In [1]:
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2
from mpmf.utils import *

In [2]:
np.random.seed(0)  # Set random seed for reproducibility
T = np.random.rand(100)
m = 5
k = 3
l = 3

In [3]:
df_motif1 = get_top_1_motif(pd.Series(T),m, l)
df_motif1

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


,top_1_motif_dist,top_1_motif_idx,top_1_motif_idx_delta,top_1_motif_point_after_1,top_1_motif_point_after_2,top_1_motif_point_after_3
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
95,0.580522,65.0,25.0,0.976459,0.468651,0.976761
96,0.918651,8.0,83.0,0.925597,0.071036,0.087129
97,0.836035,61.0,31.0,0.820993,0.097101,0.837945
98,0.846294,39.0,54.0,0.666767,0.670638,0.210383


In [4]:
df_motif2 = get_top_k_motifs(pd.Series(T), m=m, k=k, l=l)
df_motif2

/Users/cyuab/miniconda3/envs/mpmf/lib/python3.10/site-packages/stumpy/core.py:3532: RuntimeWarning: Mean of empty slice
  [np.nanmean(D_copy) - 2.0 * np.nanstd(D_copy), np.nanmin(D_copy)]
/Users/cyuab/miniconda3/envs/mpmf/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/cyuab/miniconda3/envs/mpmf/lib/python3.10/site-packages/stumpy/core.py:3532: RuntimeWarning: All-NaN slice encountered
  [np.nanmean(D_copy) - 2.0 * np.nanstd(D_copy), np.nanmin(D_copy)]
/Users/cyuab/miniconda3/envs/mpmf/lib/python3.10/site-packages/stumpy/core.py:3531: RuntimeWarning: All-NaN axis encountered
  return np.nanmax(


,top_1_motif_dist,top_2_motif_dist,top_3_motif_dist,top_1_motif_idx,top_2_motif_idx,top_3_motif_idx,top_1_motif_idx_delta,top_2_motif_idx_delta,top_3_motif_idx_delta,top_1_motif_point_after_1,top_1_motif_point_after_2,top_1_motif_point_after_3,top_2_motif_point_after_1,top_2_motif_point_after_2,top_2_motif_point_after_3,top_3_motif_point_after_1,top_3_motif_point_after_2,top_3_motif_point_after_3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.580522,NaN,NaN,65.0,NaN,NaN,25.0,NaN,NaN,0.976459,0.468651,0.976761,NaN,NaN,NaN,NaN,NaN,NaN
96,0.918651,0.936143,1.021728,8.0,70.0,25.0,83.0,21.0,66.0,0.925597,0.071036,0.087129,0.039188,0.282807,0.120197,0.264556,0.774234,0.456150
97,0.836035,1.058044,1.271959,61.0,9.0,82.0,31.0,83.0,10.0,0.820993,0.097101,0.837945,0.071036,0.087129,0.020218,0.093941,0.575946,0.929296
98,0.846294,1.021435,1.022023,39.0,2.0,20.0,54.0,91.0,73.0,0.666767,0.670638,0.210383,0.891773,0.963663,0.383442,0.639921,0.143353,0.944669


In [5]:
import datetime
print(f"This notebook was last run end-to-end on: {datetime.datetime.now()}\n")
###
###
###

This notebook was last run end-to-end on: 2025-08-17 22:42:09.925384

